### DESARROLLO LOCAL

In [12]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import sklearn
import sys

print("Versión de Python:", sys.version)
print("Versión de Pandas:", pd.__version__)
print("Versión de Numpy:", np.__version__)
print("Versión de Scikit-learn:", sklearn.__version__)

Versión de Python: 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:34:57) [MSC v.1936 64 bit (AMD64)]
Versión de Pandas: 2.0.3
Versión de Numpy: 1.23.5
Versión de Scikit-learn: 1.3.1


In [13]:
PROJECT_ID_DS = 'cmpc-innovation-cd4ml-test'

### CREATE DATASET

In [14]:
def ec(x, y, z):
    return 6*np.power(x, 3) + 5*np.power(y,2) + 10 - 7*z

In [15]:
np.random.seed(42)

# generate random features
len_values = 5000
x = np.random.random([len_values, 3])

# predict
y = ec(x[:, 0], x[:, 1], x[: ,2])

# add noise value y
y + np.random.random(len_values)

array([10.21024472, 11.06382061, 10.10734302, ...,  8.96582415,
       13.93325828, 10.74770151])

### TRAIN MODEL

In [5]:
model = LinearRegression()
model.fit(x,y)

LinearRegression()

In [6]:
y_predicted = model.predict(x)

### Evaluar model

In [7]:
r2_score(y_true = y,
         y_pred = y_predicted)

0.934927794801364

### Guardar modelo entrenado

### Guardar modelo entrenado para ser registrado en vertex models

In [8]:
import pickle
import os
from google.cloud import storage

In [9]:
"""
######## ESTA ES LA ÚNICA PARTE PARA TENER CUIDADO PARA QUE EL SCRIPT FUNCIONE TANTO EN UN ENTRENAMIENTO LOCAL COMO GCP ########
- Para que el modelo entrenado quede registrado en el menú "MODELS" de Vertex (artefacto del modelo obligatoriamente con el nombre "model.pkl") debe ser guardado en un bucket de GCS definido
por un path creado automático al enviar el job de entrenamiento en una variable de ambiente "AIP_MODEL_DIR" que solo existe cuando se corre el job de entrenamiento en Vertex. 

- Cuando se corre localmente la variable de ambiente no existe, por lo tanto, retorna un path: None

- Ejemplo path cloud: gs://discovery-vertex-ai/aiplatform-custom-training-2022-05-01-13:27:34.025/model/
"""

# Save model artifact to local filesystem (doesn't persist)
artifact_filename = 'model.pkl'
local_path = artifact_filename
with open(local_path, 'wb') as model_file:
    pickle.dump(model, model_file)
print('modelo guardado local')

# Upload model artifact to Cloud Storage - try: guardar en path var de ambiente cuando se corre en cloud y registrar en vertex // except: retornar error cuando no existe path var de ambiente
print('path del modelo a GCS - AIP_MODEL_DIR: ', os.getenv("AIP_MODEL_DIR"))
try:
    model_directory = os.environ['AIP_MODEL_DIR']
    storage_path = os.path.join(model_directory, artifact_filename)
    blob = storage.blob.Blob.from_string(storage_path, client=storage.Client())
    blob.upload_from_filename(local_path)
except Exception as e: 
    print('Error: ', str(e))
    print('MODELO NO GUARDADO EN GCS')

modelo guardado local
path del modelo a GCS - AIP_MODEL_DIR:  None
Error:  'AIP_MODEL_DIR'
MODELO NO GUARDADO EN GCS


In [10]:
# eliminar modelo guardado en el local
os.remove('model.pkl')